In [74]:
def lat_coord_to_canvas_coord(lat):
    
    # Convert latitude using Web Mercator transformation
    lat_rad = math.radians(lat)
    y = 0.5 - math.log(math.tan(math.pi/4.0 + lat_rad/2.0)) / (2.0 * math.pi)
    
    return  y

In [52]:
# Fix: The data is in regular lat/lon grid, not Web Mercator
# So use linear interpolation instead

def pixelToLatitude(row, height, min_lat, max_lat):
    """Convert pixel row to latitude using linear interpolation"""
    # Flip Y-axis: row 0 should be max_lat, row height-1 should be min_lat
    return max_lat - (row / (height - 1)) * (max_lat - min_lat)

def pixelToLongitude(col, width, min_lng, max_lng):
    """Convert pixel column to longitude using linear interpolation"""
    return min_lng + (col / (width - 1)) * (max_lng - min_lng)

# Test with your specific coordinate
test_lat = 48.154037
test_lng = -69.8376

print(f\"Testing coordinate: {test_lat}, {test_lng}\")
print(f\"Linear interpolation result: {pixelToLatitude(c, height, cur[1].min(), cur[1].max()):.6f}\")
print(f\"Expected: {test_lat}\")"

SyntaxError: unexpected character after line continuation character (<ipython-input-52-b67f740faa07>, line 17)

In [53]:
import numpy as np
import math

def yToLatitude(y):
    """
    Convert normalized Y coordinate (0-1) back to latitude (-90 to 90) using inverse Web Mercator
    
    Args:
        y: Normalized Y coordinate (0-1)
    
    Returns:
        Latitude in degrees
    """
    # Inverse Web Mercator transformation
    mercator_y = (0.5 - y) * 2.0 * math.pi  # Convert 0-1 to mercator Y range
    lat_rad = 2.0 * math.atan(math.exp(mercator_y)) - math.pi/2.0  # Inverse mercator formula
    return math.degrees(lat_rad)  # Convert radians to degrees

# Test the function
print("Testing yToLatitude function:")
print(f"y=0.0 -> lat={yToLatitude(0.0):.2f}°")
print(f"y=0.5 -> lat={yToLatitude(0.5):.2f}°") 
print(f"y=1.0 -> lat={yToLatitude(1.0):.2f}°")

def xToLongitude(x):
    return (x*360) - 180

Testing yToLatitude function:
y=0.0 -> lat=85.05°
y=0.5 -> lat=0.00°
y=1.0 -> lat=-85.05°


In [54]:
import pandas as pd

cur = pd.read_json("./mpoStleCourants_a6c0_dffb_1a94.json")
cur = pd.DataFrame(cur["table"].rows)


In [55]:
width = len(cur[1][cur[1]==cur[1][1]])
height = cur[1].unique().__len__()

In [56]:
(cur[2].max() - cur[2].min())/width

0.003999999999999995

In [57]:
print("Width, Height")
width, height

Width, Height


(1150, 1400)

In [58]:
import numpy as np
cur.loc[cur[3].isna(), 3] = 0
cur.loc[cur[4].isna(), 4] = 0


In [59]:
from sklearn.preprocessing import minmax_scale
import json
from PIL import Image
import numpy as np
from datetime import datetime

u_values = minmax_scale(cur[3])*255
v_values = minmax_scale(cur[4])*255

rgba_image = np.zeros((height, width, 4), dtype=np.uint8)
rgba_image[:,:,0] = u_values.reshape((height, width))
rgba_image[:,:,1] = v_values.reshape((height, width))
rgba_image[:,:,3] = 255

# Create PIL Image and save as PNG
img = Image.fromarray(rgba_image, 'RGBA')
img.save('current_data.png')

u_min = cur[3].min()
u_max = cur[3].max()
v_min = cur[4].min()
v_max = cur[4].max()

# Create JSON metadata file
metadata = {
    "source": "Ocean currents data",
    "date": datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
    "width": width,
    "height": height,
    "uMin": float(u_min),
    "uMax": float(u_max),
    "vMin": float(v_min),
    "vMax": float(v_max),
    "minLat": cur[1].min(),
    "maxLat": cur[1].max(),
    "latPerPixel": (cur[1].max() - cur[1].min())/height,
    "minLong": cur[2].min(),
    "maxLong": cur[2].max(),
    "longPerPixel": (cur[2].max() - cur[2].min())/width
}

with open('current_data.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("PNG and JSON files created successfully!")
print(f"Image size: {width} x {height}")
print("Image has been flipped vertically to match geographic coordinates")

PNG and JSON files created successfully!
Image size: 1150 x 1400
Image has been flipped vertically to match geographic coordinates


In [77]:
def equirectangular_to_mercator_fixed(equirect_array, min_lng, max_lng, min_lat, max_lat):
    """
    Transform an equirectangular projection image to Web Mercator projection
    Fixed version with correct Y-coordinate handling
    """
    height, width, _ = equirect_array.shape
    mercator_array = np.zeros_like(equirect_array)

    print(f"Converting {width}x{height} image from equirectangular to mercator")
    print(f"Bounds: lng({min_lng:.3f}, {max_lng:.3f}) lat({min_lat:.3f}, {max_lat:.3f})")


    min_lat_norm = lat_coord_to_canvas_coord( min_lat)
    max_lat_norm = lat_coord_to_canvas_coord( max_lat)

    def mix(a,b,c):
        return  a * (1.0 - c) + b * c

    # For each pixel in the output Mercator image
    for y in range(height):
        
        for x in range(width):
            # Convert pixel coordinates to normalized coordinates (0-1)
            norm_x = x / width
            norm_y = y / height

            norm_y = mix(min_lat_norm, max_lat_norm, norm_y)

            # In Mercator image: y=0 should be north (max_lat), y=1 should be south (min_lat)
            # Convert normalized y to latitude using inverse Web Mercator
            mercator_y = (0.5 - norm_y) * 2.0 * math.pi
            merc_lat_rad = 2.0 * math.atan(math.exp(mercator_y)) - math.pi/2.0
            merc_lat = math.degrees(merc_lat_rad)

            # Convert normalized x to longitude (linear)
            merc_lng = min_lng + norm_x * (max_lng - min_lng)

            # Clamp to data bounds
            # merc_lat = max(min_lat, min(max_lat, merc_lat))
            # merc_lng = max(min_lng, min(max_lng, merc_lng))

            pixelY = ((merc_lat - min_lat) / (max_lat - min_lat))*height
            pixelX = ((merc_lng - min_lng) / (max_lng - min_lng))*width
            
            mercator_array[y, x] = equirect_array[int(pixelY), int(pixelX)]

    return Image.fromarray(mercator_array, 'RGBA')

# Test the fixed function
print("Testing FIXED equirectangular to mercator conversion...")
mercator_img_fixed = equirectangular_to_mercator_fixed(
    rgba_image,
    cur[2].min(), cur[2].max(),  # min_lng, max_lng
    cur[1].min(), cur[1].max()   # min_lat, max_lat
)
mercator_img_fixed.save('current_data.png')
print(f"Fixed mercator image saved with dimensions: {mercator_img_fixed.size}")

Testing FIXED equirectangular to mercator conversion...
Converting 1150x1400 image from equirectangular to mercator
Bounds: lng(-72.600, -68.000) lat(46.000, 49.500)
Fixed mercator image saved with dimensions: (1150, 1400)
